In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import numpy as np
# import pandas as pd
import os
import sys

# sys.path.append('Enviroment')
# from tf_trade_enviroment import MyTradeEnv
from env.TFenv import HangmanEnvironment



from tf_agents.environments import utils
from tf_agents.environments import tf_py_environment

from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import q_rnn_network
from tf_agents.utils import common

from learningClass import learningHelper



tf.random.set_seed(12)
tf.print(tf.config.list_physical_devices('GPU') )
tf.compat.v1.enable_v2_behavior()

[]


## Read OCHLV data from the file

In [6]:
dictionary_path = "../../../properties/words_250000_train.txt"
reward_map = {
    "game_success_reward": 1.00,
    "lose_reward": -0.,
    "guess_success_reward": 0.2,
    "guess_fail_reward": -0.,
    "guess_repeat_reward": -5.,
}

## Create and validate enviroment

In [7]:
environment = HangmanEnvironment(dictionary_path,reward_map)
utils.validate_py_environment(environment, episodes=2)

print('action_spec:', environment.action_spec())
print('time_step_spec.observation:', environment.time_step_spec().observation)
print('time_step_spec.step_type:', environment.time_step_spec().step_type)
print('time_step_spec.discount:', environment.time_step_spec().discount)
print('time_step_spec.reward:', environment.time_step_spec().reward)



action_spec: BoundedArraySpec(shape=(), dtype=dtype('int32'), name='letter', minimum=0, maximum=25)
time_step_spec.observation: BoundedArraySpec(shape=(1, 30), dtype=dtype('int32'), name='game', minimum=-1, maximum=26)
time_step_spec.step_type: ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')
time_step_spec.discount: BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0)
time_step_spec.reward: ArraySpec(shape=(), dtype=dtype('float32'), name='reward')


## Create traning and validation enviroment

In [8]:
train_env = tf_py_environment.TFPyEnvironment(HangmanEnvironment(dictionary_path,reward_map))
eval_env = tf_py_environment.TFPyEnvironment(HangmanEnvironment(dictionary_path,reward_map))

## Define trainig paramters

In [9]:
learning_rate = 1e-3  

## Create q_network

In [10]:
#network configuration
input_fc_layer_params = (40,2)
lstm_size=(28,)
output_fc_layer_params=(26,)

# as we are using dictionary in our enviroment, we will create preprocessing layer
preprocessing_layers = tf.keras.layers.Flatten()
# preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)

#create a q_RNNnet
q_net = q_rnn_network.QRnnNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    preprocessing_layers=preprocessing_layers,
    # preprocessing_combiner=preprocessing_combiner,
    input_fc_layer_params=input_fc_layer_params,
    lstm_size=lstm_size,
    output_fc_layer_params=output_fc_layer_params)    

## Create the DQN-agent

In [11]:
#create optimizer
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

#create a global step coubter
#train_step_counter = tf.Variable(0)
global_step = tf.compat.v1.train.get_or_create_global_step()

#create agent
agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    #train_step_counter=train_step_counter)
    train_step_counter=global_step)

agent.initialize()

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

In [12]:
magent = learningHelper(train_env=train_env, test_env=eval_env, agent=agent, global_step=global_step, collect_episodes = 10,
eval_interval=5, verbose=0, batch_size=64, train_sequence_length=4, chkpdir='./rnn_chkp/')
magent.restore_check_point()

selected agent collect_policy


In [20]:
%%time
import warnings
warnings.filterwarnings('ignore')

# magent.train_agent(600)
magent.train_agent_with_avg_ret_condition(25, 10000, 10)

Global steps 242: Traning Loss 57.21150207519531
Global steps 243: Traning Loss 62.18061065673828
Global steps 244: Traning Loss 63.14788818359375
Global steps 245: Traning Loss 65.88768005371094
Global steps 246: Traning Loss 66.31741333007812
Global steps 247: Traning Loss 61.795188903808594
eval episodes = 10: Average Return = -20.96000099182129
Global steps 248: Traning Loss 59.772666931152344
Global steps 249: Traning Loss 61.216148376464844
Global steps 250: Traning Loss 60.34109878540039
Global steps 251: Traning Loss 55.47283935546875
Global steps 252: Traning Loss 64.84423828125
eval episodes = 10: Average Return = -20.0
Global steps 253: Traning Loss 68.26296997070312
Global steps 254: Traning Loss 60.7711181640625
Global steps 255: Traning Loss 67.30191802978516
Global steps 256: Traning Loss 66.47439575195312
Global steps 257: Traning Loss 70.41450500488281
eval episodes = 10: Average Return = -21.920000076293945
Global steps 258: Traning Loss 67.99669647216797
Global steps

In [17]:
for _ in range(10):
    magent.evaluate_agent(80)


eval episodes = 80: Average Return = -20.46000099182129
eval episodes = 80: Average Return = -20.940000534057617
eval episodes = 80: Average Return = -20.479999542236328
eval episodes = 80: Average Return = -20.46000099182129
eval episodes = 80: Average Return = -19.5
eval episodes = 80: Average Return = -19.979999542236328
eval episodes = 80: Average Return = -19.5
eval episodes = 80: Average Return = -20.479999542236328
eval episodes = 80: Average Return = -20.0
eval episodes = 80: Average Return = -19.5


In [ ]:
magent.store_check_point()

In [ ]:
magent.restore_check_point()